In [1]:
import pandas as pd
import numpy as np

In [2]:
y1_data = pd.read_csv('raw_data/2010-2011 Solar home electricity data.csv', skiprows=1)

In [3]:
y1_data.head()

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,3.78,2076,GC,1-Jul-10,0.303,0.471,0.083,0.121,0.361,...,0.495,0.54,0.406,0.543,0.495,0.216,0.378,0.128,0.078,0.125
1,1,3.78,2076,CL,1-Jul-10,1.250,1.244,1.256,0.744,0.019,...,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.075
2,1,3.78,2076,GG,1-Jul-10,0.000,0.000,0.000,0.000,0.000,...,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,3.78,2076,GC,2-Jul-10,0.116,0.346,0.122,0.079,0.120,...,0.262,1.10,1.012,0.817,0.526,0.335,0.402,0.142,0.120,0.111
4,1,3.78,2076,CL,2-Jul-10,1.238,1.238,1.256,1.250,0.169,...,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.088


In [57]:
# Filter relevant columns for a single customer
customer_id = 11
customer_data = y1_data[y1_data['Customer'] == customer_id]
customer_data = customer_data.drop(columns=['Generator Capacity', 'Postcode'])

customer_data = pd.melt(customer_data, id_vars=['Customer', 'date', 'Consumption Category'], var_name='Time')
customer_data.to_csv('customer_data.csv', index=False)

customer_data['date'] = customer_data['date'].astype(str)
zero_padded_date = []
for date in customer_data['date']:
    if len(date) < 9:
        new_date = '0' + date
        zero_padded_date.append(new_date)
    else:
        zero_padded_date.append(date)

customer_data['date'] = zero_padded_date

customer_data['Time'] = customer_data['Time'].astype(str)

customer_data['datetime'] = customer_data['date'] + ' ' + customer_data['Time']
customer_data['datetime'] = pd.to_datetime(customer_data['datetime'], format='%d-%b-%y %H:%M')
customer_data = customer_data.sort_values('datetime')
customer_data.to_csv('customer_data_new.csv', index=False)

year = customer_data['datetime'].iloc[0].year
print(type(year))
print(year)

<class 'int'>
2010


In [51]:
# Separate consumption categories
gc_data = customer_data[customer_data['Consumption Category'] == 'GC']
cl_data = customer_data[customer_data['Consumption Category'] == 'CL']
gg_data = customer_data[customer_data['Consumption Category'] == 'GG']

gc_data = gc_data.rename(columns = {'value': 'GC'})
cl_data = cl_data.rename(columns = {'value': 'CL'})
gg_data = gg_data.rename(columns = {'value': 'GG'})

gc_data = gc_data.drop(columns=['Consumption Category'])
cl_data = cl_data.drop(columns=['Consumption Category'])
gg_data = gg_data.drop(columns=['Consumption Category'])

gc_data.to_csv('data_gc.csv', index=False)
cl_data.to_csv('data_cl.csv', index=False)
gg_data.to_csv('data_gg.csv', index=False)

#if cl_data.empty == False:
#    print("not empty")
#else:
#    print("empty")

if cl_data.empty == False:
    nl_data = pd.merge(gc_data, gg_data, on=['Customer', 'date', 'Time', 'datetime'])
    nl_data = pd.merge(nl_data, cl_data, on=['Customer', 'date', 'Time', 'datetime'])
    nl_data['NL'] = nl_data['GC'] + nl_data['CL'] - nl_data['GG']
    nl_data.to_csv('data_nl.csv', index=False)
else:
    nl_data = pd.merge(gc_data, gg_data, on=['Customer', 'date', 'Time', 'datetime'])
    nl_data['NL'] = nl_data['GC'] - nl_data['GG']
    nl_data.to_csv('data_nl.csv', index=False)

In [ ]:
for i in range(1,12):
    netload_list = []
    date_list = []
    customer_data = y1_data[y1_data.Customer == i]
    print(customer_data.head())
    
    gc_data = customer_data[customer_data['Consumption Category'] == 'GC']
    gg_data = customer_data[customer_data['Consumption Category'] == 'GG']
    cl_data = customer_data[customer_data['Consumption Category'] == 'CL']
    #customer_data.to_csv('processed_data/customer_data.csv', index=False)
    count = 1
    for date in customer_data.date:
        if count == 3:
            date_list.append(date)
            print(date)
            rows = customer_data[customer_data.date == date]
            
            #print(rows)
            gc_row = rows.iloc[0, 5:53]
            cl_row = rows.iloc[1, 5:53]
            gg_row = rows.iloc[2, 5:53]

            netload_row = gc_row + cl_row - gg_row
            netload_list.append(netload_row)
            count = 1
        else:
            count += 1
        
    pd.DataFrame(netload_list).to_csv('processed_data/Netload10-11'+ '_' + str(i) + '.csv', index=False)
        

   Customer  Generator Capacity  Postcode Consumption Category      date  \
0         1                3.78      2076                   GC  1-Jul-10   
1         1                3.78      2076                   CL  1-Jul-10   
2         1                3.78      2076                   GG  1-Jul-10   
3         1                3.78      2076                   GC  2-Jul-10   
4         1                3.78      2076                   CL  2-Jul-10   

    0:30   1:00   1:30   2:00   2:30  ...  19:30  20:00  20:30  21:00  21:30  \
0  0.303  0.471  0.083  0.121  0.361  ...  0.495   0.54  0.406  0.543  0.495   
1  1.250  1.244  1.256  0.744  0.019  ...  0.000   0.00  0.000  0.000  0.000   
2  0.000  0.000  0.000  0.000  0.000  ...  0.000   0.00  0.000  0.000  0.000   
3  0.116  0.346  0.122  0.079  0.120  ...  0.262   1.10  1.012  0.817  0.526   
4  1.238  1.238  1.256  1.250  0.169  ...  0.000   0.00  0.000  0.000  0.000   

   22:00  22:30  23:00  23:30   0:00  
0  0.216  0.378  0.128 

IndexError: single positional indexer is out-of-bounds